<a href="https://www.kaggle.com/code/vtrackstar/2023-24-top-nba-midrange-scorers?scriptVersionId=197336869" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
shooting = pd.read_csv('/kaggle/input/nba-2023-24-stats/Untitled spreadsheet - Shooting and Dist.csv') 
totals = pd.read_csv('/kaggle/input/nba-2023-24-stats/Untitled spreadsheet - Totals.csv')

print(shooting.head())
print(totals.head())

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nba-2023-24-stats/Untitled spreadsheet - Shooting and Dist.csv
/kaggle/input/nba-2023-24-stats/Untitled spreadsheet - Totals.csv
   Rk            Player Pos  Age   Tm   G    MP    FG%  Dist.  Unnamed: 9  \
0   1     Caleb Houstan  SF   21  ORL  59   815  0.388   23.9         NaN   
1   2       Sam Merrill  SG   27  CLE  61  1069  0.402   23.6         NaN   
2   3        Sam Hauser  SF   26  BOS  79  1741  0.446   22.7         NaN   
3   4  Garrison Mathews  SG   27  ATL  66   992  0.456   22.6         NaN   
4   5    Doug McDermott  SF   32  TOT  64   904  0.433   21.9         NaN   

   ...  FG% for 3P  Unnamed: 23  %FGA Dunks  # Dunks  Unnamed: 26  \
0  ...       0.373          NaN       0.000        0          NaN   
1  ...       0.404          NaN       0.000        0          NaN   
2  ...       0.424          NaN       0.016        9          NaN   
3  ...       0.440          NaN       0.000        0          NaN   
4  ...       0.410          NaN       0.021      

In [2]:
print(shooting.columns)
print(totals.columns)

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'FG%', 'Dist.',
       'Unnamed: 9', '%FGA 2P', '%FGA 0-3', '%FGA 3-10', '%FGA 10-16',
       '%FGA 16-3P', '%FGA 3P', 'Unnamed: 16', 'FG% for 2P', 'FG% for 0-3',
       'FG% for 3-10', 'FG% for 10-16', 'FG% for 16-3P', 'FG% for 3P',
       'Unnamed: 23', '%FGA Dunks', '# Dunks', 'Unnamed: 26', '%3PA Corner',
       '3P% Corner', 'Unnamed: 29', 'Att Heaves', '# Heaves'],
      dtype='object')
Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA\n▼',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')


In [3]:
totals.rename(columns={'FGA\n▼': 'FGA'}, inplace=True)

df1 = shooting[['Player', 'Pos', 'Age', 'G', 'MP', '%FGA 2P', '%FGA 0-3', '%FGA 3-10', '%FGA 10-16', '%FGA 16-3P', '%FGA 3P', 'FG% for 2P', 'FG% for 0-3', 'FG% for 3-10', 'FG% for 10-16', 'FG% for 16-3P']]
df2 = totals[['Player', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%']]

merged = pd.merge(df1, df2, on='Player', how='inner')
print(merged.head())
print(merged.columns)

             Player Pos  Age   G    MP  %FGA 2P  %FGA 0-3  %FGA 3-10  \
0     Caleb Houstan  SF   21  59   815    0.063     0.019      0.019   
1       Sam Merrill  SG   27  61  1069    0.099     0.041      0.038   
2        Sam Hauser  SF   26  79  1741    0.167     0.072      0.048   
3  Garrison Mathews  SG   27  66   992    0.146     0.044      0.031   
4    Doug McDermott  SF   32  64   904    0.194     0.099      0.042   

   %FGA 10-16  %FGA 16-3P  ...  FG% for 0-3  FG% for 3-10  FG% for 10-16  \
0       0.015       0.010  ...        0.500         0.750          1.000   
1       0.015       0.005  ...        0.500         0.467          0.000   
2       0.027       0.020  ...        0.675         0.407          0.467   
3       0.053       0.018  ...        0.800         0.000          0.583   
4       0.025       0.028  ...        0.607         0.333          0.571   

   FG% for 16-3P   FG  FGA    FG%  2P  2PA    2P%  
0          0.000   80  206  0.388   8   13  0.615  
1     

In [4]:
# Calculate the estimated mid-range attempts from 10 feet - 3P
merged['Estimated FGA 10-3P'] = merged['FGA'] * (merged['%FGA 10-16'] + merged['%FGA 16-3P'])

# Calculate the percentage of FGA that are from 10 feet - 3P
merged['%FGA 10-3P'] = merged['%FGA 10-16'] + merged['%FGA 16-3P']

# Calculate the FG percentage of shots that are from 10 feet - 3P
merged['FG% for 10-3P'] = merged['FG% for 10-16'] * merged['FG% for 16-3P']

merged['Estimated FGA 10-3P'] = pd.to_numeric(merged['Estimated FGA 10-3P'])


print(merged.head())

             Player Pos  Age   G    MP  %FGA 2P  %FGA 0-3  %FGA 3-10  \
0     Caleb Houstan  SF   21  59   815    0.063     0.019      0.019   
1       Sam Merrill  SG   27  61  1069    0.099     0.041      0.038   
2        Sam Hauser  SF   26  79  1741    0.167     0.072      0.048   
3  Garrison Mathews  SG   27  66   992    0.146     0.044      0.031   
4    Doug McDermott  SF   32  64   904    0.194     0.099      0.042   

   %FGA 10-16  %FGA 16-3P  ...  FG% for 16-3P   FG  FGA    FG%  2P  2PA  \
0       0.015       0.010  ...          0.000   80  206  0.388   8   13   
1       0.015       0.005  ...          0.000  158  393  0.402  15   39   
2       0.027       0.020  ...          0.636  249  558  0.446  52   93   
3       0.053       0.018  ...          0.750  103  226  0.456  18   33   
4       0.025       0.028  ...          0.500  123  284  0.433  29   55   

     2P%  Estimated FGA 10-3P  %FGA 10-3P  FG% for 10-3P  
0  0.615                5.150       0.025       0.000000 

In [5]:
# Calculate the Midrange Score
merged['Midrange Score'] = merged['Estimated FGA 10-3P'] * merged['FG% for 10-3P']

# Sort the DataFrame by the new score to find the best scorers
top_mid = merged.sort_values(by='Midrange Score', ascending=False)

top15_mid = top_mid.head(15)

# Display the updated DataFrame with the new score
print(top15_mid[['Player', 'Estimated FGA 10-3P', 'FG% for 10-3P', 'Midrange Score']])

                      Player  Estimated FGA 10-3P  FG% for 10-3P  \
106             Kevin Durant              630.404       0.260230   
94              Devin Booker              498.510       0.260072   
142            DeMar DeRozan              650.400       0.196980   
181  Shai Gilgeous-Alexander              510.041       0.250160   
120           Brandon Ingram              466.803       0.236544   
109          Dejounte Murray              380.380       0.265189   
121            Jalen Brunson              459.792       0.201924   
129            Kawhi Leonard              398.566       0.212500   
46               Paul George              336.192       0.225936   
68               Luka Dončić              386.568       0.186524   
111               Cam Thomas              361.760       0.189420   
139          Cade Cunningham              303.420       0.225456   
159           Paolo Banchero              409.437       0.166295   
105             Kyrie Irving              291.79